## Minimal working example

Script creation for the LDclumping.ipynb

In [21]:
%save MWE_ldclump.sh -f

# Defining bash variables for the different paths,

output=output
bfile=~/UKBB_GWAS_DEV/workflow/data/genotypes.bed
bgenFile=`echo ~/UKBB_GWAS_DEV/workflow/data/ukb_imp_chr{1..22}_v3.bgen`
sampleFile=~/UKBB_GWAS_DEV/workflow/data/imputed_genotypes.sample
sumstatsFile=~/UKBB_GWAS_DEV/workflow/data/phenotypes_BMI.fastGWA.snp_stats.gz
unrelated_samples=~/UKBB_GWAS_DEV/workflow/data/imputed_genotypes.1000unrelatedindivid
ld_sample_size=100
clump_field=P
clump_p1=0.05
clump_p2=1
clump_r2=0.2
clump_kb=1000
clump_annotate=BP
numThreads=20
job_size=1

#Running the workflow for BMI trait

sos dryrun ~/UKBB_GWAS_DEV/workflow/LD_Clumping.ipynb \
    --cwd $output \
    --bfile $bfile \
    --bgenFile $bgenFile \
    --sampleFile $sampleFile \
    --sumstatsFile $sumstatsFile \
    --unrelated_samples $unrelated_samples \
    --ld_sample_size $ld_sample_size \
    --clump_field $clump_field \
    --clump_p1 $clump_p1 \
    --clump_p2 $clump_p2 \
    --clump_r2 $clump_r2 \
    --clump_kb $clump_kb \
    --clump_annotate $clump_annotate \
    --numThreads $numThreads \
    --job_size $job_size
    -q localhost \
    -s build &> 061620-sos-LD-MWE.log

Cell content saved to MWE_ldclump.sh, use option -r to also execute the cell.